In [42]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import networkx as nx
import plotly.express as px
from plotly.offline import plot
import plotly.graph_objs as go
import plotly.express as px
import plotly.graph_objects as go
#import ptitprince                     #Raincloud
import plotly.io as pio
import plotly.io as pio
pio.renderers.default='browser'
import numpy as np
from pynput.mouse import Listener

In [44]:
# Excel-Datei laden
file_path = 'Liste_Version_2.xlsx'  # Ersetze dies durch den tatsächlichen Pfad
df = pd.read_excel(file_path)
df.columns = df.columns.str.strip()

# Überprüfe, ob die erforderlichen Spalten existieren
required_columns = ['Bereich', 'Fachbereich', 'Name_des_Tools']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"Die erforderlichen Spalten {required_columns} sind nicht in der Datei vorhanden.")

# Graphen erstellen
G = nx.Graph()

# Kanten zwischen Bereich, Fachbereich und Name des Tools hinzufügen
for _, row in df[required_columns].dropna().iterrows():
    G.add_edge(row['Bereich'], row['Fachbereich'])
    G.add_edge(row['Fachbereich'], row['Name_des_Tools'])

# 3D Layout für den Graphen
pos = nx.spring_layout(G, dim=3)

# Knotenpositionen extrahieren
x_vals = [pos[node][0] for node in G.nodes]
y_vals = [pos[node][1] for node in G.nodes]
z_vals = [pos[node][2] for node in G.nodes]

# Sicherstellen, dass der Punkt "DH" existiert
if 'DH' not in G.nodes:
    raise ValueError('Der Knoten "DH" existiert nicht im Graphen.')


# Position von "DH"
dh_pos = pos['DH']
dh_x, dh_y, dh_z = dh_pos

# Plotly-Spur für Knoten
node_trace = go.Scatter3d(
    x=x_vals, y=y_vals, z=z_vals,
    mode='markers+text',
    text=list(G.nodes),
    textposition='top center',
    marker=dict(size=6, color='lightblue', opacity=0.8)
)

# Plotly-Spur für Kanten
edge_trace = []
for edge in G.edges():
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    edge_trace.append(go.Scatter3d(
        x=[x0, x1, None], y=[y0, y1, None], z=[z0, z1, None],
        mode='lines',
        line=dict(color='gray', width=1),
        hoverinfo='none'
    ))

# Traces kombinieren
fig = go.Figure(data=[*edge_trace, node_trace])


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"}
    }

# Animation für Rotation um "DH"
frames = []
angle_step = 1  # Schrittweite für jeden Frame
for angle in range(0, 360, angle_step):
    rad = np.radians(angle)
    eye_x = dh_x + 2 * np.cos(rad)
    eye_y = dh_y + 2 * np.sin(rad)
    eye_z = dh_z + 0.5  # Z-Achse bleibt konstant
    
    frames.append(dict(
        layout=dict(
            scene_camera=dict(
                eye=dict(x=eye_x, y=eye_y, z=eye_z)
            )
        )
    ))

# Hinzufügen der Frames für die Animation
fig.frames = frames

# Animation konfigurieren
fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(label="Play",
                      method="animate",
                      args=[None, dict(frame=dict(duration=50, redraw=True),
                                       fromcurrent=True, mode='immediate')]),
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },                
                                       ]
    )]
)


# Layout anpassen
fig.update_layout(
    title='Tools der Digital Humanities',
    title_x=0.5,
    showlegend=False,
    scene=dict(
        xaxis=dict(showbackground=False, showticklabels=False, title=None),
        yaxis=dict(showbackground=False, showticklabels=False, title=None),
        zaxis=dict(showbackground=False, showticklabels=False, title=None),
        camera=dict(
            eye=dict(x=1.25, y=1.25, z=1.25)  # Initiale Kameraansicht
        )
    )
)



# Graph anzeigen und speichern
fig.show()
fig.write_html("fig4.html")


RuntimeError: threads can only be started once

In [ ]:
fig = go.Figure(data=[edge_trace, node_trace], layout=layout)
fig.show()